# BReact SDK Demo

This notebook demonstrates how to use the BReact SDK to interact with various services.

## Table of Contents
1. [Client Initialization](#Client-Initialization)
2. [Text Summarization Example](#Text-Summarization-Example)
3. [Email Generator Example](#Email-Generator-Example)
4. [Information Tracker](#Information-Tracker)
5. [Using Generic Services](#Using-Generic-Services)
6. [Cleanup](#Cleanup)


In [2]:
# Now import and use the client
from breact_sdk import BReactClient

# Initialize client
client = await BReactClient.create(
    api_key=<apikey>
)

2025-02-03 12:06:34,928 - breact_sdk - INFO - Creating new BReactClient instance...
2025-02-03 12:06:34,929 - breact_sdk - INFO - Initializing BReactClient...
2025-02-03 12:06:34,929 - breact_sdk - INFO - Initializing HTTP client with base URL: http://localhost:8000
2025-02-03 12:06:34,929 - breact_sdk.http - INFO - Initializing HTTP client for http://localhost:8000
2025-02-03 12:06:35,097 - breact_sdk.http - INFO - HTTP client initialized successfully
2025-02-03 12:06:35,097 - breact_sdk - INFO - BReactClient base initialization completed
2025-02-03 12:06:35,098 - breact_sdk - INFO - Initializing services...
2025-02-03 12:06:35,098 - breact_sdk - INFO - Fetching available services from API...
2025-02-03 12:06:35,099 - breact_sdk.http - INFO - Making GET request to /api/v1/services
2025-02-03 12:06:35,411 - breact_sdk - INFO - Found 11 services
2025-02-03 12:06:35,412 - breact_sdk - INFO - BReactClient initialization completed successfully


## Text Summarization Example – Summarizing a Sample Text

In this section, we use the BReact SDK to generate a concise summary of a sample text. The parameters such as `summary_type`, `model_id`, and `options` control the summarization behavior.

In [4]:
sample_text = """
Artificial Intelligence (AI) has transformed various industries, from healthcare to finance.
Machine learning algorithms can now process vast amounts of data to identify patterns and make predictions.
Natural Language Processing has enabled computers to understand and generate human-like text.
Computer Vision systems can recognize objects and faces with remarkable accuracy.
These advancements have led to improved automation and decision-making capabilities.
"""

result = await client.ai_summarize(
    text=sample_text,
    summary_type="brief",
    model_id="mistral-small",
    options={
        "temperature": 0.3,
        "max_tokens": 500
    }
)

if result.status == "completed" and result.result:
    print("Summary:")
    print(result.result)
else:
    print(f"Error: {result.error or 'Unknown error'}")

2025-02-03 12:06:35,429 - breact_sdk - INFO - Summarizing text...
2025-02-03 12:06:35,430 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/aisummary/summarize with polling
2025-02-03 12:06:35,431 - breact_sdk.http - INFO - Making POST request to /api/v1/services/aisummary/summarize
2025-02-03 12:06:35,466 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/20
2025-02-03 12:06:36,495 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/20
2025-02-03 12:06:37,554 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/20


Summary:
status='success' result=None error=None summary='Artificial Intelligence (AI) has significantly impacted numerous sectors, including healthcare and finance, through the use of machine learning algorithms that analyze large data sets to discern patterns and make predictions. Notable AI advancements include Natural Language Processing, allowing computers to understand and produce human-like text, and Computer Vision systems, which accurately recognize objects and faces. These developments have enhanced automation and decision-making processes.' metadata={'model': 'mistral-small', 'created_at': '2025-02-03T12:06:36.679146', 'original_length': 60, 'summary_length': 63, 'processing_time': '2025-02-03T12:06:36.679146', 'instance_id': 2621007212880, 'usage': {'prompt_tokens': 183, 'total_tokens': 267, 'completion_tokens': 84}}


## Email Generator Example – Analyzing and Responding to an Email Thread

This section demonstrates how to analyze an email thread for sentiment, key points, action items, and response urgency. Based on the analysis, a professional email response is generated using the BReact SDK.

In [6]:
# First, let's analyze the email thread
analysis_result = await client.analyze_email_thread(
    email_thread=[{
        "sender": "client@example.com",
        "recipient": "support@company.com",
        "subject": "Project Timeline Question",
        "content": """
        Hi Support Team,
        
        I'm wondering about the timeline for our current project. We seem to be running behind
        schedule on the database implementation phase. Could you provide an update on when
        we can expect this to be completed?
        
        Also, will this delay affect our final delivery date?
        
        Thanks,
        John
        """,
        "timestamp": "2024-03-20T10:00:00Z"
    }],
    analysis_type=["sentiment", "key_points", "action_items", "response_urgency"],
    model_id="mistral-small"
)

if analysis_result.status == "completed" and analysis_result.result:
    analysis = analysis_result.result.analysis  # Get the analysis dictionary
    print("Thread Analysis Results:")
    print(f"\nSentiment: {analysis['overall_sentiment']}")
    print("\nMain Points Discussed:")
    for point in analysis['main_points_discussed']:
        print(f"- {point}")
    print("\nAction Items:")
    for item in analysis['action_items']:
        print(f"- {item['task']} (Urgency: {item['urgency']})")
    print(f"\nResponse Urgency: {analysis['response_urgency']}")

    # Now generate a response based on the analysis
    response_result = await client.generate_email_response(
        email_thread=[{
            "sender": "client@example.com",
            "recipient": "support@company.com",
            "subject": "Project Timeline Question",
            "content": """
            Hi Support Team,
            
            I'm wondering about the timeline for our current project. We seem to be running behind
            schedule on the database implementation phase. Could you provide an update on when
            we can expect this to be completed?
            
            Also, will this delay affect our final delivery date?
            
            Thanks,
            John
            """,
            "timestamp": "2024-03-20T10:00:00Z"
        }],
        tone="professional",
        style_guide={
            "language": "en",
            "max_length": 200,
            "signature": "Best regards,\nSupport Team",
            "greeting_style": "formal"
        },
        key_points=[
            "Address timeline concerns",
            "Explain database implementation status",
            "Discuss impact on final delivery"
        ],
        model_id="mistral-small",
        options={
            "temperature": 0.3,
            "max_tokens": 1000
        }
    )
    
    if response_result.status == "completed" and response_result.result:
        response = response_result.result.response  # Get the response dictionary
        print("\nGenerated Response:")
        print(f"Subject: {response['subject']}")
        print("\nContent:")
        print(response['content'])
        
        if response['suggestions']:
            print("\nAlternative Suggestions:")
            for suggestion in response['suggestions']:
                print(f"- {suggestion}")
    else:
        print(f"\nError generating response: {response_result.error or 'Unknown error'}")
else:
    print(f"\nError analyzing thread: {analysis_result.error or 'Unknown error'}")

2025-02-03 12:06:37,603 - breact_sdk - INFO - Analyzing email thread...
2025-02-03 12:06:37,604 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/email_response/analyze_thread with polling
2025-02-03 12:06:37,604 - breact_sdk.http - INFO - Making POST request to /api/v1/services/email_response/analyze_thread
2025-02-03 12:06:37,638 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/21
2025-02-03 12:06:38,883 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/21
2025-02-03 12:06:39,938 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/21
2025-02-03 12:06:40,987 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/21
2025-02-03 12:06:41,035 - breact_sdk - INFO - Generating email response...
2025-02-03 12:06:41,036 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/email_response/generate_response with polling
2025-02-03 12:06:41,036 - breact_sdk.http - INFO - Maki

Thread Analysis Results:

Sentiment: neutral

Main Points Discussed:
- Project timeline question
- Concern about delay in database implementation phase
- Request for update on completion of database implementation phase
- Potential impact of delay on final delivery date

Action Items:
- Provide update on database implementation phase timeline (Urgency: high)
- Evaluate impact of delay on final delivery date (Urgency: high)

Response Urgency: high


2025-02-03 12:06:42,097 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/22
2025-02-03 12:06:43,154 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/22
2025-02-03 12:06:44,210 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/22
2025-02-03 12:06:45,261 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/22



Generated Response:
Subject: Re: Project Timeline Question

Content:
{
  "subject": "Response to Project Timeline Question",
  "content": "Dear John,\n\nI hope this message finds you well. I appreciate your concern regarding the project timeline.\n\nAt this stage, our team is working diligently to complete the database implementation phase. We anticipate that this work will be finalized by 2024-03-27T10:00:00Z, which is a slight delay from our initial projections.\n\nI regret any inconvenience this may cause. However, I want to assure you that we are committed to delivering high-quality work and will make every effort to minimize any potential impact on the overall project timeline.\n\nOnce the database implementation is complete, we will promptly proceed to the next phases of the project to ensure we meet the final delivery date. We will keep you updated on our progress and notify you of any changes.\n\nThank you for your understanding and patience.\n\nBest regards,\n[Your Name]",
  

## Information Tracker – Extracting Structured Information from Unstructured Text

Here we provide a JSON schema to extract structured medical information from a free-text note. The `extract_information` method uses the schema and additional context (`current_info`) to produce a structured analysis.

In [8]:
# Example of using the Information Tracker service
medical_note = """
Patient reports severe headache lasting for 3 days, accompanied by nausea and sensitivity to light. 
Pain level is described as 8/10. Symptoms worsen with physical activity. 
No prior history of migraines, but family history shows mother had similar symptoms.
"""

# Define the schema for medical symptom extraction
schema = {
    "type": "object",
    "properties": {
        "primary_symptom": {
            "type": "string",
            "enum": ["headache", "nausea", "dizziness", "fatigue", "pain"]
        },
        "duration": {
            "type": "string"
        },
        "pain_level": {
            "type": "string"
        },
        "associated_symptoms": {
            "type": "array",
            "items": {
                "type": "string"
            }
        },
        "aggravating_factors": {
            "type": "array",
            "items": {
                "type": "string"
            }
        },
        "family_history": {
            "type": "object",
            "properties": {
                "present": {
                    "type": "boolean"
                },
                "details": {
                    "type": "string"
                }
            },
            "required": ["present"]
        }
    },
    "required": ["primary_symptom", "duration", "pain_level"]
}

# Extract information using the new method
result = await client.extract_information(
    text=medical_note,
    schema=schema,
    update_type="medical_symptoms",
    current_info={
        "previous_symptoms": ["mild headache"]
    }
)

if result.status == "completed":
    if result.result.status == "error":
        print(f"Error extracting information: {result.result.error}")
    else:
        print("Analysis Results:")
        analysis = result.result.result
        
        print(f"\nPrimary Symptom: {analysis['primary_symptom']}")
        print(f"Duration: {analysis['duration']}")
        print(f"Pain Level: {analysis['pain_level']}")
        
        if 'associated_symptoms' in analysis:
            print("\nAssociated Symptoms:")
            for symptom in analysis['associated_symptoms']:
                print(f"- {symptom}")
        
        if 'aggravating_factors' in analysis:
            print("\nAggravating Factors:")
            for factor in analysis['aggravating_factors']:
                print(f"- {factor}")
        
        if 'family_history' in analysis:
            print("\nFamily History:")
            print(f"Present: {analysis['family_history']['present']}")
            if 'details' in analysis['family_history']:
                print(f"Details: {analysis['family_history']['details']}")
else:
    print(f"Error: {result.error or 'Unknown error'}")

2025-02-03 12:06:45,323 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/information_tracker/extract_information with polling
2025-02-03 12:06:45,324 - breact_sdk.http - INFO - Making POST request to /api/v1/services/information_tracker/extract_information
2025-02-03 12:06:45,358 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/23
2025-02-03 12:06:46,537 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/23
2025-02-03 12:06:47,593 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/23


Analysis Results:

Primary Symptom: headache
Duration: 3 days
Pain Level: 8/10

Associated Symptoms:
- nausea

Aggravating Factors:
- physical activity

Family History:
Present: True
Details: mother had similar symptoms


## Using Generic Services – Invoking Custom Service Endpoints

This section demonstrates how to call any available service endpoint using the `execute_service` method. This method allows flexibility to interact with services that may not have dedicated SDK implementations.

In [10]:
# Test webscraper service using the correct endpoint name
result = await client.execute_service(
    service_id="webscraper",
    endpoint="scrape",  # Changed from scrape_with_llm to scrape
    params={
        "url": "https://example.com",
        "extraction_description": "Extract the main article content",
        "model_id": "gpt-4o",  # optional
        "options": {  # optional
            "wait_until": "networkidle",
            "timeout": 30,
            "text_mode": True
        }
    }
)

print(f"Status: {result.status if hasattr(result, 'status') else 'N/A'}")
print("\nResult:")
print(result)

2025-02-03 12:06:47,654 - breact_sdk.http - INFO - Executing POST request to /api/v1/services/webscraper/scrape with polling
2025-02-03 12:06:47,655 - breact_sdk.http - INFO - Making POST request to /api/v1/services/webscraper/scrape
2025-02-03 12:06:47,832 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/24
2025-02-03 12:06:48,875 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/24
2025-02-03 12:06:51,298 - breact_sdk.http - INFO - Making GET request to /api/v1/services/result/24


Status: completed

Result:
status='completed' result=ServiceResult(status='success', result={'main_article_content': 'This domain is for use in illustrative examples in documents. You may use this domain in literature without prior coordination or asking for permission.'}, error=None, metadata={'url': 'https://example.com', 'created_at': '2025-02-03T12:06:47.832664', 'processing_time': 2.396634, 'model': 'gpt-4o'}) error=None message=None service='webscraper' endpoint='scrape' created_at=datetime.datetime(2025, 2, 3, 12, 6, 47, 820856) process_info=None


## Cleanup – Closing the Client

It is important to close the client after use to free up resources and properly terminate connections.

In [12]:
await client.close()
print("Client closed successfully!")

2025-02-03 12:06:51,349 - breact_sdk - INFO - Closing client and cleaning up resources...
2025-02-03 12:06:51,349 - breact_sdk.http - INFO - Closing HTTP client
2025-02-03 12:06:51,350 - breact_sdk - INFO - Client closed successfully


Client closed successfully!
